In [5]:
'''
1. get & clean data
    tech:
        small: NASDAQ:NTGR, NYSE:PLT
        medium: NYSE:SMI
        large: NASDAQ:AAPL, NASDAQ:AMZN
    non-tech:
        small: NYSE:HRI, NYSE:DF
        medium: NASDAQ:UHAL
        larege: NYSE: BRK.A, NYSE:XOM
2. OLS with 3 classic factor
3. get google search trend
4. regress on different sectors of revenue
5. automate the process

x. next task: ML training
'''
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols
# http://www.statsmodels.org/devel/generated/statsmodels.regression.linear_model.RegressionResults.html

In [12]:
# stock data; 20120706-20170630
odf = pd.read_csv(r"FACTORS - source_text.csv")
odf = odf[1:]
df = odf.iloc[:,[1,3,5,7,9,11,13,15,17,19]].astype('float')
# data preprocessing
print(df.tail())
df = df.pct_change()[1:]*100
date = odf.iloc[:,0].str.split(' ').str[0]
date = pd.to_datetime(date, format="%m/%d/%Y")[1:]
df.insert(0, "date", date)
df.set_index('date', inplace=True)
df.columns = [x.replace('.', '_') for x in df.columns]
print(df.tail())

       AAPL     AMZN   SMI   NTGR    PLT     BRK.A    XOM    UHAL    HRI  \
257  155.45  1006.73  5.16  43.35  55.33  249660.0  79.50  356.45  38.82   
258  148.98   978.31  4.95  45.20  56.09  254965.0  82.13  358.11  38.39   
259  142.27   987.71  5.68  43.35  53.92  256450.1  83.49  350.95  37.37   
260  146.28  1003.74  5.37  43.95  53.05  251800.0  81.61  340.72  37.54   
261  144.02   968.00  5.70  43.10  52.31  254700.0  80.73  366.06  39.32   

        DF  
257  18.33  
258  17.87  
259  18.01  
260  17.71  
261  17.00  
                AAPL      AMZN        SMI      NTGR       PLT     BRK_A  \
date                                                                      
2017-06-02  1.197839  1.099640  -3.189493  2.000000  4.278176  0.450632   
2017-06-09 -4.162110 -2.823001  -4.069767  4.267589  1.373577  2.124890   
2017-06-16 -4.503960  0.960841  14.747475 -4.092920 -3.868782  0.582472   
2017-06-23  2.818584  1.622946  -5.457746  1.384083 -1.613501 -1.813257   
2017-06-30 -1.5

In [13]:
# 3 factors; 20120706-20170526
fff = pd.read_csv(r"fff.csv")
fff.rename(columns={"Unnamed: 0":"date"}, inplace=True)
fff.date = pd.to_datetime(fff.date, format="%Y%m%d")
fff.set_index('date', inplace=True)
fff = fff[1:]
fff.columns = [x.replace('-','_') for x in fff.columns]
# print(fff.tail())
# print(fff.shape)

In [14]:
ndf = df.iloc[:255]
rdf = pd.concat([ndf,fff], axis=1)

model_dict = {}
pvalues_dict = {}
for i in range(df.shape[1]):
    key = rdf.columns[i]
    model_dict[key] = ols(rdf.columns[i] + " ~ PREM+SMB+HML", rdf).fit()
    print(key, model_dict[key].pvalues[model_dict[key].pvalues < 0.05])
    print(key, model_dict[key].params)

AAPL PREM    1.242006e-16
HML     2.705530e-03
dtype: float64
AAPL Intercept   -0.014107
PREM         1.112746
SMB         -0.232734
HML         -0.569673
dtype: float64
AMZN PREM    2.876508e-18
HML     3.226147e-07
dtype: float64
AMZN Intercept    0.342040
PREM         1.214473
SMB          0.083022
HML         -1.015702
dtype: float64
SMI PREM    0.000491
dtype: float64
SMI Intercept    0.411792
PREM         0.789238
SMB         -0.417270
HML          0.186649
dtype: float64
NTGR PREM    1.961228e-10
SMB     8.261697e-03
dtype: float64
NTGR Intercept   -0.129992
PREM         1.007599
SMB          0.626527
HML          0.223054
dtype: float64
PLT PREM    7.813322e-29
SMB     2.053124e-07
dtype: float64
PLT Intercept   -0.113950
PREM         1.327292
SMB          0.866342
HML          0.040227
dtype: float64
BRK_A PREM    1.167202e-44
HML     2.541068e-09
dtype: float64
BRK_A Intercept    0.047957
PREM         0.816798
SMB         -0.130819
HML          0.438546
dtype: float64
XOM Int

In [15]:
print(model_dict['DF'].params.iloc[1])

0.814737966222


In [298]:
fig = plt.figure(figsize=(18,16))
df.plot.area(stacked=False)
